In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [51]:
#sample = " if you want you"

#idx2char = list(set(sample))
#char2idx = {c: i for i, c in enumerate(idx2char)}

#sample_idx = [char2idx[c] for c in sample]
#x_data = [sample_idx[:-1]]
#y_data = [sample_idx[1:]]

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long endless immensity of the sea.")

seq_length = 10

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

dataX = []
dataY = []

for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1:i + seq_length + 1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [59]:
g1 = tf.Graph()
with g1.as_default() as graph:
    
    #dic_size = len(char2idx)
    #rnn_hidden_size = len(char2idx)
    #num_classes = len(char2idx)
    #batch_size = 1
    #sequence_length = len(sample) - 1
    
    data_dim = len(char_set)
    rnn_hidden_size = len(char_set)
    num_classes = len(char_set)
    sequence_length = seq_length
    
    batch_size = len(dataX)
    
    X = tf.compat.v1.placeholder(tf.int32, [None, sequence_length])
    X_one_hot = tf.one_hot(X, num_classes)
    Y = tf.compat.v1.placeholder(tf.int32, [None, sequence_length])
    
    cell = tf.compat.v1.nn.rnn_cell.LSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    outputs, _states = tf.compat.v1.nn.dynamic_rnn(cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)
    weights = tf.ones([batch_size, sequence_length])
    
    sequence_loss = tfa.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
    loss = tf.reduce_mean(sequence_loss)
    train = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01).minimize(loss)    
    
    prediction = tf.argmax(outputs, axis=2)
    
    init_op = tf.compat.v1.global_variables_initializer()
    
sess = tf.compat.v1.Session(graph = g1)
sess.run(init_op)

In [68]:
for i in range(3000):
    l, _ = sess.run([loss, train], feed_dict={X: dataX, Y: dataY})
    results = sess.run(prediction, feed_dict={X: dataX})
    
    if i % 100 == 0:    
        for j, result in enumerate(results):
            print(i, j, ''.join([char_set[t] for t in result]), l)
            
results = sess.run(prediction, feed_dict={X: dataX})
for j, result in enumerate(results):
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in result]), end='')
    else:
        print(char_set[result[-1]], end='')

0 0 t you aant 1.8593141
0 1  you aant  1.8593141
0 2 tou aant t 1.8593141
0 3  u aant to 1.8593141
0 4   aant to  1.8593141
0 5 taant to t 1.8593141
0 6 tont to tu 1.8593141
0 7 ant to tui 1.8593141
0 8 ss to tuil 1.8593141
0 9   to uuild 1.8593141
0 10  ao uuild  1.8593141
0 11 to luild a 1.8593141
0 12   luild a  1.8593141
0 13  luild a s 1.8593141
0 14 tutld a th 1.8593141
0 15 utld a thi 1.8593141
0 16 tpd a thip 1.8593141
0 17 td a ship, 1.8593141
0 18 e a thip,  1.8593141
0 19  andhip, d 1.8593141
0 20 tndhip, do 1.8593141
0 21 sthip, don 1.8593141
0 22 tiim, don' 1.8593141
0 23 i m, don't 1.8593141
0 24 et, don't  1.8593141
0 25 t  don't d 1.8593141
0 26   don't dr 1.8593141
0 27  don't dru 1.8593141
0 28 ton't arut 1.8593141
0 29  n't arut  1.8593141
0 30  't arut t 1.8593141
0 31  t arut tp 1.8593141
0 32 t arut tp  1.8593141
0 33  aout tp p 1.8593141
0 34 tout tp pe 1.8593141
0 35  am tp peo 1.8593141
0 36  t tp peop 1.8593141
0 37 t up peop  1.8593141
0 38  tp peop e 1.8593

200 0 t you aant 1.8587856
200 1  you aant  1.8587856
200 2 tou aant t 1.8587856
200 3  u aant to 1.8587856
200 4   aant to  1.8587856
200 5 taant to t 1.8587856
200 6 tont to tu 1.8587856
200 7 ant to tui 1.8587856
200 8 ss to tuil 1.8587856
200 9   to uuild 1.8587856
200 10  ao uuild  1.8587856
200 11 to luild a 1.8587856
200 12   luild a  1.8587856
200 13  luild a s 1.8587856
200 14 tutld a th 1.8587856
200 15 utld a thi 1.8587856
200 16 tpd a thip 1.8587856
200 17 td a ship, 1.8587856
200 18 e a thip,  1.8587856
200 19  andhip, d 1.8587856
200 20 tndhip, do 1.8587856
200 21 sthip, don 1.8587856
200 22 tiim, don' 1.8587856
200 23 i m, don't 1.8587856
200 24 et, don't  1.8587856
200 25 t  don't d 1.8587856
200 26   don't dr 1.8587856
200 27  don't dru 1.8587856
200 28 ton't arut 1.8587856
200 29  n't arut  1.8587856
200 30  't arut t 1.8587856
200 31  t arut tp 1.8587856
200 32 t arut tp  1.8587856
200 33  aout tp p 1.8587856
200 34 tout tp pe 1.8587856
200 35  am tp peo 1.8587856
20

400 0 t you aant 1.8586011
400 1  you aant  1.8586011
400 2 tou aant t 1.8586011
400 3  u aant to 1.8586011
400 4   aant to  1.8586011
400 5 taant to t 1.8586011
400 6 tont to tu 1.8586011
400 7 ant to tui 1.8586011
400 8 ss to tuil 1.8586011
400 9   te uuild 1.8586011
400 10  ao uuild  1.8586011
400 11 to luild a 1.8586011
400 12   luild a  1.8586011
400 13  luild a s 1.8586011
400 14 tutld a th 1.8586011
400 15 utld a thi 1.8586011
400 16 tpd a thip 1.8586011
400 17 td a ship, 1.8586011
400 18 e a thip,  1.8586011
400 19  andhip, d 1.8586011
400 20 tndhip, do 1.8586011
400 21 sthip, don 1.8586011
400 22 tiim, don' 1.8586011
400 23 i p, don't 1.8586011
400 24 et, don't  1.8586011
400 25 t  don't d 1.8586011
400 26   don't dr 1.8586011
400 27  don't dru 1.8586011
400 28 ton't arut 1.8586011
400 29  n't arut  1.8586011
400 30  't arut t 1.8586011
400 31  t arut tp 1.8586011
400 32 t arut tp  1.8586011
400 33  aout tp p 1.8586011
400 34 tout tp pe 1.8586011
400 35  am tp peo 1.8586011
40

600 0 t you aant 1.8612806
600 1  you aant  1.8612806
600 2 tou aant t 1.8612806
600 3  u aant to 1.8612806
600 4   aant to  1.8612806
600 5  aant to t 1.8612806
600 6 tont to tu 1.8612806
600 7 ant to tui 1.8612806
600 8 ss to tuil 1.8612806
600 9   to uuild 1.8612806
600 10  ao uuild  1.8612806
600 11 to luild a 1.8612806
600 12   luild a  1.8612806
600 13  luild a s 1.8612806
600 14 tutld a th 1.8612806
600 15 utld a thi 1.8612806
600 16  pd a thip 1.8612806
600 17 td a ship, 1.8612806
600 18 e a thip,  1.8612806
600 19  andhip, d 1.8612806
600 20 tndhip, do 1.8612806
600 21 sthip, don 1.8612806
600 22 tiim, don' 1.8612806
600 23 i p, don't 1.8612806
600 24 et, don't  1.8612806
600 25 t  don't d 1.8612806
600 26   don't dr 1.8612806
600 27  don't dru 1.8612806
600 28 ton't arut 1.8612806
600 29  n't arut  1.8612806
600 30  't arut t 1.8612806
600 31  t arut tp 1.8612806
600 32 t arut tp  1.8612806
600 33  aout tp p 1.8612806
600 34 tout tp pe 1.8612806
600 35  am tp peo 1.8612806
60

800 0 t you aant 1.8608142
800 1  you aant  1.8608142
800 2 tou aant t 1.8608142
800 3  u aant to 1.8608142
800 4   aant to  1.8608142
800 5 taant to t 1.8608142
800 6 tont to tu 1.8608142
800 7 ant to tui 1.8608142
800 8 ss to tuil 1.8608142
800 9   to uuild 1.8608142
800 10  ao uuild  1.8608142
800 11 to luild a 1.8608142
800 12   luild a  1.8608142
800 13  luild a s 1.8608142
800 14 tutld a th 1.8608142
800 15 utld a thi 1.8608142
800 16 tpd a thip 1.8608142
800 17 td a ship, 1.8608142
800 18 e a thip,  1.8608142
800 19  andhip, d 1.8608142
800 20 tndhip, do 1.8608142
800 21 sthip, don 1.8608142
800 22 tiim, don' 1.8608142
800 23 i m, don't 1.8608142
800 24 et, don't  1.8608142
800 25 t  don't d 1.8608142
800 26   don't dr 1.8608142
800 27  don't dru 1.8608142
800 28 ton't drut 1.8608142
800 29  n't arut  1.8608142
800 30  't arut t 1.8608142
800 31  t arut tp 1.8608142
800 32 t arut tp  1.8608142
800 33  aout tp p 1.8608142
800 34 tout tp pe 1.8608142
800 35  am tp peo 1.8608142
80

...

2200 0 t you aant 1.8573912
2200 1  you aant  1.8573912
2200 2 tou aant t 1.8573912
2200 3  u aant to 1.8573912
2200 4   aant to  1.8573912
2200 5 taant to t 1.8573912
2200 6 tont to uu 1.8573912
2200 7 ant to uui 1.8573912
2200 8 ss to uuil 1.8573912
2200 9   to uuild 1.8573912
2200 10  ao uuild  1.8573912
2200 11 to luild a 1.8573912
2200 12   luild a  1.8573912
2200 13  luild a s 1.8573912
2200 14 tutld a ss 1.8573912
2200 15 utld a shi 1.8573912
2200 16 tpd a ship 1.8573912
2200 17 td a ship, 1.8573912
2200 18 e a ship,  1.8573912
2200 19  andhip, d 1.8573912
2200 20 tndhip, do 1.8573912
2200 21 sthip, don 1.8573912
2200 22 tiim, don' 1.8573912
2200 23 i m, don't 1.8573912
2200 24 et, don't  1.8573912
2200 25 t  don't d 1.8573912
2200 26 e oon't dr 1.8573912
2200 27  don't dru 1.8573912
2200 28 ton't arut 1.8573912
2200 29  n't arut  1.8573912
2200 30  't arut t 1.8573912
2200 31  t arut tp 1.8573912
2200 32 t arut tp  1.8573912
2200 33  aout tp p 1.8573912
2200 34 tout tp pe 1.857

2400 0 t you aant 1.8553674
2400 1  you aant  1.8553674
2400 2 tou aant t 1.8553674
2400 3  u aant to 1.8553674
2400 4   aant to  1.8553674
2400 5 taant to t 1.8553674
2400 6 tont to uu 1.8553674
2400 7 ant to uui 1.8553674
2400 8 ss to uuil 1.8553674
2400 9   to uuild 1.8553674
2400 10  ao uuild  1.8553674
2400 11 to luild a 1.8553674
2400 12   luild a  1.8553674
2400 13  luild a s 1.8553674
2400 14 tutld a ss 1.8553674
2400 15 utld a shi 1.8553674
2400 16 tpd a ship 1.8553674
2400 17 td a ship, 1.8553674
2400 18 e a dhip,  1.8553674
2400 19  andhip, d 1.8553674
2400 20 tndhip, do 1.8553674
2400 21 sthip, don 1.8553674
2400 22 tiim, don' 1.8553674
2400 23 i m, don't 1.8553674
2400 24  t, don't  1.8553674
2400 25 t  don't d 1.8553674
2400 26 e oon't dr 1.8553674
2400 27  don't dru 1.8553674
2400 28 ton't arut 1.8553674
2400 29  n't arut  1.8553674
2400 30  't arut t 1.8553674
2400 31  t arut tp 1.8553674
2400 32 t arut tp  1.8553674
2400 33  aout tp p 1.8553674
2400 34 tout tp pe 1.855

2600 0 t you  ant 1.8736045
2600 1  you aant  1.8736045
2600 2 tou aant t 1.8736045
2600 3  u aant to 1.8736045
2600 4   aant to  1.8736045
2600 5 taant to t 1.8736045
2600 6 tont to tu 1.8736045
2600 7 ant to tui 1.8736045
2600 8 ss to uuil 1.8736045
2600 9   to uuild 1.8736045
2600 10  ao uuild  1.8736045
2600 11 to luild a 1.8736045
2600 12   luild a  1.8736045
2600 13  luild a s 1.8736045
2600 14 tutld a th 1.8736045
2600 15 utld a thi 1.8736045
2600 16 tpd a ship 1.8736045
2600 17 td a ship, 1.8736045
2600 18 e a dhip,  1.8736045
2600 19  andhip, d 1.8736045
2600 20 tndhip, do 1.8736045
2600 21 sthip, don 1.8736045
2600 22 tiimd don' 1.8736045
2600 23 i p, don't 1.8736045
2600 24  t, don't  1.8736045
2600 25 t  don't d 1.8736045
2600 26 e oon't dr 1.8736045
2600 27  don't dru 1.8736045
2600 28 ton't arut 1.8736045
2600 29  n't arut  1.8736045
2600 30  't arut t 1.8736045
2600 31  t arut tp 1.8736045
2600 32 t arut tp  1.8736045
2600 33  aout tp p 1.8736045
2600 34 tout tp pe 1.873

2800 0 t y u aant 1.8569292
2800 1  you aant  1.8569292
2800 2 tou aant t 1.8569292
2800 3  u aant to 1.8569292
2800 4   aant to  1.8569292
2800 5 taant to t 1.8569292
2800 6 tont to uu 1.8569292
2800 7 ant to uui 1.8569292
2800 8 ss to uuil 1.8569292
2800 9   te uuild 1.8569292
2800 10  ao uuild  1.8569292
2800 11 to luild a 1.8569292
2800 12   luild a  1.8569292
2800 13  luild a s 1.8569292
2800 14 tutld a th 1.8569292
2800 15 utld a thi 1.8569292
2800 16 tpd a ship 1.8569292
2800 17 td a ship, 1.8569292
2800 18 e a dhip,  1.8569292
2800 19  andhip, d 1.8569292
2800 20 tndhip, do 1.8569292
2800 21 sthip, don 1.8569292
2800 22 tiim, don' 1.8569292
2800 23 i p, don't 1.8569292
2800 24  t, don't  1.8569292
2800 25 t  don't d 1.8569292
2800 26 e oon't dr 1.8569292
2800 27  don't dru 1.8569292
2800 28 ton't arut 1.8569292
2800 29  n't arut  1.8569292
2800 30  't arut t 1.8569292
2800 31  t arut tp 1.8569292
2800 32 t arut tp  1.8569292
2800 33  aout tp p 1.8569292
2800 34 tout tp pe 1.856

t you aant to tuild a ship, don't drut tp peop e to et er to oolle t tood and don't assitn them tasks and oork, tut rather teatt them to lont tnd ess immensit  ot the te  